In [1]:
from glob import glob
from bs4 import BeautifulSoup
import os
import numpy as np; import pandas as pd

## PARSING XML FILE
### Read DOC filenames

### parsing all text from each xml 

## Reading xlsx with text examples

In [2]:
#df_keys = pd.DataFrame(columns=('filename', 'text', 'keywords'))
fname = "/Users/cavaste/GoogleDriveCGnal/Projects/Dedalus/dedalus_text_sample_used.xlsx"

df_text = pd.read_excel(fname, thousands = ',', parse_cols = 'A:C')
df_text.head()

,Filename,ID,Text
0,table_export_DATA44e8cdc0-0159-1000-8001-c0a80...,DH_AC605550710_1500,"Primary, Malignant neoplasm of prostate"
1,table_export_DATA44e8cdc0-0159-1000-8001-c0a80...,DH_AC605550710_1527,"Secondary, Anemia in chronic kidney disease"
2,table_export_DATA44e8cdc0-0159-1000-8001-c0a80...,DH_AC605550710_1521,"Secondary, Iron deficiency anemia, unspecified"
3,table_export_DATA44e8d2c1-0159-1000-8002-c0a80...,DH_MC876686273_17,"Primary, Stage II, Malignant neoplasm of recto..."
4,table_export_DATA44e8d4a6-0159-1000-8003-c0a80...,DH_Cb671607614_1393,"Primary, Malign Neoplasm Ovarian"


In [3]:
from sklearn.feature_extraction import stop_words
stopwords_ = set(stop_words.ENGLISH_STOP_WORDS)

from nltk.corpus import stopwords

stopwords_.update(stopwords.words('english'))
#stopwords_.update(stopwords.words('italian'))

stopwords_.update(["patient", "primary", "secondary"])

In [4]:
df_text["Text"] = df_text["Text"].apply(lambda x: x.lower())

## Threshold for text length

In [5]:
lengthText_thr = 25
groupbyID = "Filename"
ratio_value = 0.9

### Counting the number of words in each text

In [6]:
df_text.insert(3, "Word Count", df_text["Text"].apply(lambda x: len(x.split())))

### Selecting short texts

In [7]:
df_text_short = df_text[df_text["Word Count"] < lengthText_thr].iloc[:,[0,1,2,3]]

## 1. POS TAGGING + FILTER 
### to prepare full text search for drugs and procedures detection

### POS tagging

In [8]:
import nltk
df_text.insert(4, "Pos Tag", df_text["Text"].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x))))

In [9]:
df_text_pos_grouped = df_text.groupby(df_text[groupbyID]).agg(lambda x: x.tolist())

In [10]:
df_text_pos_grouped["Pos Tag"] = df_text_pos_grouped["Pos Tag"].apply(lambda x: [item for sublist in x for item in sublist])

In [11]:
pos_filter=['FW','RB','JJ','NN','NNS','NNP','NNPS']
df_text_pos_grouped.insert(5, "Filtered Text", 
                    df_text_pos_grouped["Pos Tag"].apply(lambda x: [word[0] for word in x if word[1] in pos_filter and word[1] not in stopwords_]))

In [12]:
df_text_pos_grouped["Pos Tag"].ix[16]

[(u'[', 'JJ'),
 (u'additional', 'JJ'),
 (u'reason', 'NN'),
 (u']', 'JJ'),
 (u'transition', 'NN'),
 (u'into', 'IN'),
 (u'care', 'NN'),
 (u'-', ':'),
 (u'the', 'DT'),
 (u'patient', 'NN'),
 (u'is', 'VBZ'),
 (u'transitioning', 'VBG'),
 (u'into', 'IN'),
 (u'care', 'NN'),
 (u'and', 'CC'),
 (u'a', 'DT'),
 (u'summary', 'NN'),
 (u'of', 'IN'),
 (u'care', 'NN'),
 (u'was', 'VBD'),
 (u'reviewed', 'VBN'),
 (u'.', '.'),
 (u'acute', 'JJ'),
 (u'abdominal', 'JJ'),
 (u'pain', 'NN'),
 (u'chronic', 'JJ'),
 (u'rlq', 'NN'),
 (u'pain', 'NN')]

## 2. LONG TEXT: KEYWORD EXTRACTION

### Selecting long texts

In [13]:
df_text_long = df_text[df_text["Word Count"] >= lengthText_thr].iloc[:,[0,1,2,3]]

### Keyword extraction from each text by using graph

In [14]:
df_text_long["Text"].head()

20    hemorrhoids - patient with a long history of h...
25    *post-operative - this is a 65-year-old female...
51    breast mass, right : follow up by phone in sev...
52    breast lump - ms. pruznak is a very pleasant 2...
54    follow up - mr herbert presented for a followu...
Name: Text, dtype: object

In [15]:
from gensim.summarization import keywords

In [16]:
keys_list = []

for doc in (df_text_long["Text"]):
    keys = keywords(doc,
                    WINDOW_SIZE=4,
                    split=False,
                    ratio=ratio_value,
                    scores=True,
                    lemmatize=True)
#                    pos_filter=['NN','JJ','NNS','NNP','NNPS','FW','RB','CD'])
    
    keys = [key for key in keys if key[0] not in stopwords_][:30]    
    keys_list.append(keys) 

In [17]:
df_text_long.insert(4,"Keywords",keys_list)# = [filenames[i], doc, keys] 

In [46]:
df_text_long["Filename"].iloc[6]

u'table_export_DATA44e94695-0159-1000-8025-c0a80a0a809c.ldr'

In [48]:
pippo = 1
df_text_long["Keywords"].iloc[6], df_text_long["Text"].iloc[6]

([(u'discharged', array([ 0.28319952])),
  (u'post', array([ 0.26656862])),
  (u'status', array([ 0.25745553])),
  (u'fevers shaking', array([ 0.23573585])),
  (u'laparoscopic appendectomy', array([ 0.22599174])),
  (u'today', array([ 0.22313578])),
  (u'female', array([ 0.21825869])),
  (u'returned', array([ 0.21318233])),
  (u'chills', array([ 0.2075917])),
  (u'appetite', array([ 0.2075917])),
  (u'wound staples', array([ 0.20588263])),
  (u'postoperative interval', array([ 0.17584309])),
  (u'normal', array([ 0.1709396])),
  (u'year', array([ 0.15816796])),
  (u'ready', array([ 0.14946641])),
  (u'clinic', array([ 0.14946641]))],
 u'post-operative - this is a 56-year-old female who is status post laparoscopic appendectomy. she is having now wishes in the postoperative interval. no fevers shaking chills. appetite has returned to normal. today her wound staples were removed and she is ready to be discharged from the clinic.')

## 3. SHORT TEXT: aggregation + keyword extraction

### Aggregating short texts

In [19]:
df_text_short_grouped = df_text_short.groupby(df_text_short[groupbyID]).agg(lambda x: x.tolist())

In [20]:
df_text_short_grouped.reset_index(inplace = True)

In [21]:
df_text_short_grouped["Text"] = df_text_short_grouped["Text"].apply(lambda x: '. '.join(x))#.replace(",",""))

In [22]:
keys_list = []

for i,doc in enumerate(df_text_short_grouped["Text"]):
#    print(i)
#    print(doc)
    keys = keywords(doc,
                    WINDOW_SIZE=4,
                    split=False,
                    ratio=ratio_value, #ratio_value,
                    scores=True,
                    lemmatize=True)#,
#                    pos_filter=['NN','JJ','NNS','NNP','NNPS'])
    
    keys = [key for key in keys if key[0] not in stopwords_][:30]    
    keys_list.append(keys) 
    

In [23]:
keys_list

[[(u'anemia', array([ 0.38715096])),
  (u'iron deficiency', array([ 0.30657429])),
  (u'disease', array([ 0.28012902])),
  (u'prostate', array([ 0.25875068])),
  (u'chronic kidney', array([ 0.25507076])),
  (u'malignant neoplasm', array([ 0.25178733]))],
 [(u'malignant neoplasm', array([ 0.51310514])),
  (u'stage', array([ 0.39711063])),
  (u'junction', array([ 0.28111611]))],
 [(u'oxycontin', array([ 0.35150152])),
  (u'tablet', array([ 0.30324843])),
  (u'ovarian', array([ 0.2901983])),
  (u'encounter', array([ 0.28181717])),
  (u'malign neoplasm', array([ 0.26535609])),
  (u'resistant', array([ 0.25573098])),
  (u'release', array([ 0.25573098])),
  (u'antineoplastic', array([ 0.22576577]))],
 [(u'unspecified lump', array([ 0.58100055])),
  (u'breast', array([ 0.40303642]))],
 [(u'nodes', array([ 0.42620212])),
  (u'unspecified', array([ 0.35715472])),
  (u'enlarged lymph', array([ 0.33550346])),
  (u'cause', array([ 0.31425331])),
  (u'helicobacter pylori', array([ 0.30739328])),
  

In [24]:
df_text_short_grouped.insert(4,"Keywords",keys_list)

## merging df with keywords

In [25]:
df_text_keyword_list = [df_text_short_grouped, df_text_long]
df_text_keyword = pd.concat(df_text_keyword_list, ignore_index = True)

In [26]:
df_text_keyword.head()

,Filename,ID,Text,Word Count,Keywords
0,table_export_DATA44e8cdc0-0159-1000-8001-c0a80...,"[DH_AC605550710_1500, DH_AC605550710_1527, DH_...","primary, malignant neoplasm of prostate. secon...","[5, 6, 5]","[(anemia, [0.387150955162]), (iron deficiency,..."
1,table_export_DATA44e8d2c1-0159-1000-8002-c0a80...,[DH_MC876686273_17],"primary, stage ii, malignant neoplasm of recto...",[8],"[(malignant neoplasm, [0.513105137814]), (stag..."
2,table_export_DATA44e8d4a6-0159-1000-8003-c0a80...,"[DH_Cb671607614_1393, DH_DK39300751_1651, MH_D...","primary, malign neoplasm ovarian. secondary, e...","[4, 5, 6, 6]","[(oxycontin, [0.35150151853]), (tablet, [0.303..."
3,table_export_DATA44e8d665-0159-1000-8004-c0a80...,[DH_KP69546999_1216],"primary, unspecified lump in breast",[5],"[(unspecified lump, [0.581000554075]), (breast..."
4,table_export_DATA44e8d742-0159-1000-8005-c0a80...,"[DH_AC112325761_833, Reason For Visit]","secondary, enlarged lymph nodes, unspecified. ...","[5, 11]","[(nodes, [0.426202115995]), (unspecified, [0.3..."


In [29]:
pippo = 2
df_text_keyword.Keywords[pippo], df_text_keyword["Text"].ix[pippo]

([(u'oxycontin', array([ 0.35150152])),
  (u'tablet', array([ 0.30324843])),
  (u'ovarian', array([ 0.2901983])),
  (u'encounter', array([ 0.28181717])),
  (u'malign neoplasm', array([ 0.26535609])),
  (u'resistant', array([ 0.25573098])),
  (u'release', array([ 0.25573098])),
  (u'antineoplastic', array([ 0.22576577]))],
 u'primary, malign neoplasm ovarian. secondary, encounter for antineoplastic chemotherapy. oxycontin 20 mg tablet,crush resistant,extended release. oxycontin 30 mg tablet,crush resistant,extended release')

In [ ]:
df_text_keyword_grouped = df_text_keyword.groupby(df_text_keyword[groupbyID]).agg(lambda x: x.tolist())
df_text_keyword_grouped["ID"] = df_text_keyword_grouped["ID"].apply(lambda x: [item for sublist in x for item in sublist])
df_text_keyword_grouped["Keywords"] = df_text_keyword_grouped["Keywords"].apply(lambda x: [item for sublist in x for item in sublist])

## Merging keywords and filtered Tag

In [ ]:
df_text_keyword_grouped.head()

In [ ]:
df_text_pos_grouped.head()

In [ ]:
df_text_pos_keywords = pd.concat([df_text_keyword_grouped, df_text_pos_grouped["Filtered Text"]], axis=1)

In [ ]:
df_text_pos_keywords.head()

## Exporting df grouped short to csv

In [ ]:
output_filename = "/Users/cavaste/GoogleDriveCGnal/Projects/Dedalus/grouped_docs_keywords_ratio" + str(ratio_value) + "thrLength" + str(lengthText_thr) + ".csv"
df_text_pos_keywords.to_csv(path_or_buf = output_filename, sep=';',
                               index=False,
                               encoding="utf8")

In [ ]:
def replace_with_separator(text, separator, regexs):
    replacement = r"\1" + separator + r"\2"
    result = text
    for regex in regexs:
        result = regex.sub(replacement, result)
    return result

In [ ]:
import re
PAT_ALPHABETIC = re.compile('(((?![\d])\w)+)', re.UNICODE)

In [ ]:
text = "pippo, andava sembre molto velocemente. Un giorno si schianta, ahi che dolore! disse\n molto mi sono interrogato\n su cio"

In [ ]:
AB_ACRONYM_LETTERS = re.compile("([a-zA-Z])\.([a-zA-Z])\.", re.UNICODE)

In [ ]:
from gensim.utils import to_unicode
text = to_unicode(text)
text_without_acronyms = replace_with_separator(text, "", [AB_ACRONYM_LETTERS])

In [ ]:
for match in PAT_ALPHABETIC.finditer(text_without_acronyms): # tiene solo parole, butta via punteggiatura
    print(match.group())